In [15]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [16]:
from pathlib import Path
import sys

root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")

# IMPORTANT: add src, not project root
sys.path.append(str(root / "src"))

# Sanity test: this should now work
import faster_rcnn
print("faster_rcnn package at:", faster_rcnn.__file__)


faster_rcnn package at: /Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src/faster_rcnn/__init__.py


In [3]:
from pathlib import Path
import sys

root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")
sys.path.append(str(root / "src"))

from faster_rcnn.data.data_loaders_faster_rcnn import build_train_val_loaders

ckpt_path   = root / "runs" / "frcnn_polyp" / "epoch_009.pth"
train_json  = root / "src" / "artifacts" / "train.json"
val_json    = root / "src" / "artifacts" / "val.json"
roots_map   = root / "src" / "artifacts" / "roots_map.json"
images_root = root / "src" / "faster_rcnn" / "data" / "center_stratified"

train_loader, val_loader = build_train_val_loaders(
    train_json=str(train_json),
    val_json=str(val_json),
    roots_map=str(roots_map),
    images_root=str(images_root),
    img_size=832,           # only affects TRAIN augs
    train_augs="light",
    batch_size=2,
    num_workers=0,
)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src/faster_rcnn/augmentations/presets_faster_rcnn.py:289: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  A.Affine(  # replaces ShiftScaleRotate


In [4]:
images, targets = next(iter(val_loader))
img = images[0]
_, H_t, W_t = img.shape
print("val tensor size:", (W_t, H_t))
print("image_ids:", [int(t["image_id"]) for t in targets])


val tensor size: (1440, 1080)
image_ids: [1, 2]


/Users/rebekaheichberg/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [7]:
#!/usr/bin/env python3
from pathlib import Path
import sys
import torch

# Add your src folder to Python path
SRC = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src")
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from faster_rcnn.data.data_loaders_faster_rcnn import build_train_val_loaders
from faster_rcnn.data.coco_eval_faster_rcnn import coco_map
from faster_rcnn.models.faster_rcnn_model import build_fasterrcnn

# --- paths ---
root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")
ckpt_path  = root / "runs" / "frcnn_polyp" / "epoch_009.pth"
train_json = root / "src" / "artifacts" / "train.json"
val_json   = root / "src" / "artifacts" / "val.json"
roots_map  = root / "src" / "artifacts" / "roots_map.json"
images_root = root / "src" / "faster_rcnn" / "data" / "center_stratified"

# --- setup model ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = build_fasterrcnn(num_classes=2, pretrained=False)
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state["model"])
model.to(device).eval()

# --- build validation loader ---
_, val_loader = build_train_val_loaders(
    train_json=str(train_json),
    val_json=str(val_json),
    roots_map=str(roots_map),
    images_root=str(images_root),
    img_size=832,        # used for TRAIN; val uses build_val_augs
    train_augs="light",  # doesn't affect val
    batch_size=2,
    num_workers=0,
)

# --- run COCO eval ---
stats = coco_map(
    model,
    val_loader,
    device,
    coco_gt_json=str(val_json),
    verbose=True,        # final COCO summary
    interim_eval=True,   # set False if it feels too slow / spammy
    print_every=1        # you can bump this to 5/10/etc for fewer interim evals
)
print(stats)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


COCO eval:   0%|                                                             | 0/20 [03:02<?, ?it/s]

[1/20] processed 24 detections
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).


IndexError: list index out of range

In [3]:
#!/usr/bin/env python3
from pathlib import Path
import sys
import torch

# Add your src folder to Python path
SRC = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src")
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from faster_rcnn.data.data_loaders_faster_rcnn import build_train_val_loaders
from faster_rcnn.data.coco_eval_faster_rcnn import coco_map
from faster_rcnn.models.faster_rcnn_model import build_fasterrcnn

# --- paths ---
root = Path("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures")
ckpt_path  = root / "runs" / "frcnn_polyp" / "epoch_009.pth"
train_json = root / "src" / "artifacts" / "train.json"
val_json   = root / "src" / "artifacts" / "val.json"
roots_map  = root / "src" / "artifacts" / "roots_map.json"
images_root = root / "src" / "faster_rcnn" / "data" / "center_stratified"

# --- setup model ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = build_fasterrcnn(num_classes=2, pretrained=False)
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state["model"])
model.to(device).eval()

# --- build validation loader ---
_, val_loader = build_train_val_loaders(
    train_json=str(train_json),
    val_json=str(val_json),
    roots_map=str(roots_map),
    images_root=str(images_root),
    img_size=832,        # affects TRAIN only; val uses build_val_augs
    train_augs="light",
    batch_size=2,
    num_workers=0,
)

# --- run COCO eval ---
metrics = coco_map(
    model,
    val_loader,
    device,
    coco_gt_json=str(val_json),
    verbose=True,     # prints COCO table at the end
    log_every=1,      # print per batch; set 5/10 if too chatty
)

print("\nFinal metrics:")
print(
    f"  mAP@[.5:.95] = {metrics['mAP_50_95']:.4f}\n"
    f"  mAP@0.5      = {metrics['mAP_50']:.4f}\n"
    f"  Recall       = {metrics['recall']:.4f}\n"
    f"  Precision    = {metrics['precision']:.4f}"
)


/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src/faster_rcnn/augmentations/presets_faster_rcnn.py:289: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  A.Affine(  # replaces ShiftScaleRotate


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


COCO eval:   0%|                                                             | 0/20 [00:00<?, ?it/s]/Users/rebekaheichberg/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
COCO eval:   5%|██▌                                                 | 1/20 [03:04<58:28, 184.66s/it]

[batch 1/20] cumulative detections: 24
   top scores this batch: [0.98159605 0.94629735 0.73693657]


COCO eval:  10%|█████▏                                              | 2/20 [06:04<54:37, 182.09s/it]

...

[batch 18/20] cumulative detections: 395
   top scores this batch: [0.979199  0.6697963 0.4877494]


COCO eval:  95%|████████████████████████████████████████████████▍  | 19/20 [57:24<03:00, 180.35s/it]

[batch 19/20] cumulative detections: 406
   top scores this batch: [0.74590045 0.46069196 0.20074451]


COCO eval: 100%|███████████████████████████████████████████████████| 20/20 [58:40<00:00, 176.02s/it]

[batch 20/20] cumulative detections: 421
   top scores this batch: [0.71712273 0.59059113 0.49000624]
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.730
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.541
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.021
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.081
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.511
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.404
 Average Recall     (AR) @[ IoU=0.50:0.95 |

In [4]:
pwd

'/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/notebooks'

In [12]:
from PIL import Image
from faster_rcnn.deploy.inference_service import *

model = load_model()
img = Image.open("/Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures/src/faster_rcnn/data/center_stratified/C3/images/val/c3s000p4_0d17_C3_EndoCV2021_00180.jpg")
preds = run_inference(model, img, score_thr=0.3)
print(preds)
img_with_boxes = draw_boxes(img, preds)
img_with_boxes.show()


[{'id': 0, 'box': [655.8455810546875, 626.7365112304688, 766.4119262695312, 754.4381103515625], 'score': 0.981596052646637}, {'id': 1, 'box': [422.2256164550781, 485.1336669921875, 530.6296997070312, 574.904052734375], 'score': 0.9462974667549133}, {'id': 2, 'box': [786.2018432617188, 235.1293182373047, 836.5023193359375, 281.91192626953125], 'score': 0.7369361519813538}, {'id': 3, 'box': [777.5625610351562, 581.8145141601562, 848.9518432617188, 653.7422485351562], 'score': 0.5375287532806396}]


In [14]:
!streamlit run ../src/faster_rcnn/deploy/streamlit_app.py



      👋 Welcome to Streamlit!

      If you'd like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  ^C


In [ ]:
cd /Users/rebekaheichberg/Desktop/git-re/colorectal_imaging/Architectures
PYTHONPATH=src:$PYTHONPATH streamlit run src/faster_rcnn/deploy/streamlit_app.py
